In [1]:
import pandas as pd
import zipfile
import numpy as np
import matplotlib.pyplot as plt
import json
import os


from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import mutual_info_classif
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
		accuracy_score as acc_score,
		recall_score   as recall,
		confusion_matrix,
        mean_squared_error as mse, 
        r2_score
		)
from sklearn.decomposition import PCA


#MODELS
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
from sklearn import tree
from sklearn.linear_model import Perceptron
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
#import xgboost as xgb
#xgb_regressor = xgb.XGBRegressor()
from sklearn.neural_network import MLPRegressor  # Use MLPRegressor for regression
#from minepy import MINE

In [ ]:
#Path
#Insert your local repo path to file
path_home = "D:\Archivos\Social Data Challenge\data\grids\quarter_samples"

csv_100 = pd.read_csv(path_home + '\grid_100_htmp.csv')
csv_500 = pd.read_csv(path_home + '\grid_500_htmp.csv')

In [ ]:
csv_100.head()

In [ ]:
x =  csv_100.iloc[:, 1:-2]
y_crime23q1 = csv_100['crime23q1']
y_sample23q1 = csv_100['sample23q1']

#scaler = StandardScaler()
#X_scaled = scaler.fit_transform(X)
#x_1=pd.DataFrame(X_scaled, columns = X.columns)

#X_scaled[X_scaled.sample22q2 != 0]
#x[x.crime22q4 >0]

In [ ]:
# Split the dataset into training and testing sets

#y = y_crime23q1
y= y_sample23q1
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Maximal Information Coefficient Analysis

In [ ]:
!pip install --upgrade pip
!pip install --upgrade wheel
!pip install --upgrade setuptools
!pip3 install minepy

In [ ]:
from minepy import MINE

In [ ]:
#x.iloc[:,2]
x.columns[1]

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(x)

In [ ]:
x_scaled = pd.DataFrame(X_scaled, columns = x.columns)

In [ ]:
x_scaled

In [ ]:
# Initialize the MINE object

# Create a dictionary to store MIC scores for each column
mic_scores = {}

mine = MINE(alpha=0.6, c=15, est="mic_approx")
subset_data = x_scaled.copy()
            
# Iterate through columns in the subset_data DataFrame
for column_name in subset_data.columns:
    # Calculate MIC score for the column
    mine.compute_score(subset_data[column_name], y)  # Replace 'target_column' with your target column
    mic_score = mine.mic()

    # Store the MIC score in the dictionary
    mic_scores[column_name] = mic_score

# Sort the dictionary by MIC scores in descending order
sorted_mic_scores = dict(sorted(mic_scores.items(), key=lambda item: item[1], reverse=True))

count = 1
# Print the sorted MIC scores
for column_name, mic_score in sorted_mic_scores.items():
    print(f"{count}. {column_name}: {mic_score:.4f}")
    count +=1

Since the Maximal Information Coefficient variables belong to the 'sample' columns, we create a sub sataset with only those columns to test the accuracy of the Linear regression models

In [ ]:
scaled= True
mic = False
pca_flag = False
pca_components = 1
df = csv_100.copy()

x =  df.iloc[:, 1:-2]

if scaled :
    scaler = StandardScaler()
    x =  pd.DataFrame(scaler.fit_transform(x), columns = x.columns)


if mic:
    mic_cols= [col for col in x.columns if col.startswith('sample')]
    x= x[mic_cols]

if pca_flag:
    pca = PCA(n_components=pca_components)
    pca.fit(x)
    x=pd.DataFrame(pca.transform(x))
    
#x_to_use= x_scaled

y_crime23q1 = df['crime23q1']
y_sample23q1 = df['sample23q1']
y =  y_crime23q1

###########
###########

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)


In [ ]:
x.head()

In [ ]:
models = {
#    'Linear Regression': LinearRegression(),
    'Random Forest Regressor': RandomForestRegressor()#,
    #'Support Vector Regressor': SVR(),
    #'Perceptron': Perceptron(),
    #'Neural Network': MLPRegressor()
    # Add more regressor models and hyperparameters here
}

# Define a parameter grid for each model
param_grids = {
    'Linear Regression': {},
    'Random Forest Regressor': {
        'n_estimators': [500],
        'max_depth': [30]
    }        
        # Add more hyperparameters to tune here
    #,
    'Support Vector Regressor': {
       'kernel': ['linear', 'rbf'],
       'C': [0.1, 1, 10]#,
        # Add more hyperparameters to tune here
    },
    'Perceptron': {
        'penalty': [None, 'l2', 'l1', 'elasticnet'],
        'alpha': [0.0001, 0.001, 0.01],
        # Add more hyperparameters to tune here
    },
#    'Neural Network': {
#        'hidden_layer_sizes': [(50,), (100, 50), (100, 100, 50)],
#        'activation': ['relu', 'tanh'],
#        'alpha': [0.0001, 0.001, 0.01],
        # Add more hyperparameters to tune here
    #}
    # Add parameter grids for other models
}



In [ ]:
# Initialize a dictionary to store the best models and their scores
best_models = {}
best_scores = {}
best_parameters = {}


# Iterate through the models and perform grid search
for model_name, model in models.items():
    param_grid = param_grids[model_name]
    grid_search = GridSearchCV(model, param_grid, cv=5, scoring='neg_mean_squared_error')
    grid_search.fit(X_train, y_train)
    
    # Get the best model and its score
    best_model = grid_search.best_estimator_
    best_score = -grid_search.best_score_  # Negative MSE
    best_param = grid_search.best_params_

    
    # Store the best model and score
    best_models[model_name] = best_model
    best_scores[model_name] = best_score
    best_parameters[model_name] = best_param
    

# Display the results
results_df = pd.DataFrame({'Model': list(best_models.keys()), 
                           'Best Parameters': list(best_parameters.values()),
                           'Best MSE Score':  list(best_scores.values())})

results_df.sort_values(by='Best MSE Score', ascending=True, inplace=True)
print(results_df)

print(best_models)
print(best_scores)
print(best_parameters)


In [ ]:
results_df